In [10]:
import urllib.request as request
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA

In [11]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [12]:
request.urlretrieve("https://github.com/chatgpt-kr/openai-api-tutorial/raw/main/ch07/2020_%EA%B2%BD%EC%A0%9C%EA%B8%88%EC%9C%B5%EC%9A%A9%EC%96%B4%20700%EC%84%A0_%EA%B2%8C%EC%8B%9C.pdf",
                    "data-files/economy-dictionary.pdf")

('data-files/economy-dictionary.pdf',
 <http.client.HTTPMessage at 0x1c7374bc310>)

In [13]:
%%time
# pdf 파일의 텍스트 정보를 읽어서 페이지 단위로 반환
loader = PyPDFLoader('data-files/economy-dictionary.pdf')
pages = loader.load_and_split()

CPU times: total: 16.9 s
Wall time: 17 s


In [14]:
print( len(pages) )
# print( pages[0].page_content )
# print( pages[12].page_content ) # 0 ~ 12번 문서는 목차
# print( pages[-1].page_content ) # 마지막 문서는 기타 정보

pages2 = pages[13:-1] # 불필요한 문서(페이지) 제거
print(len(pages2))

366
352


In [15]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

In [16]:
documents = text_splitter.split_documents(pages2)

In [17]:
len(documents)

646

In [18]:
# 분할된 문서들을 vectordb에 저장 (여기서는 Chroma)

chroma_db = Chroma.from_documents(documents, OpenAIEmbeddings()) # 메모리에 저장
# chroma_db = Chroma.from_documents(documents, OpenAIEmbeddings(), 
#                                   persist_directory="vectordb/chroma.economy.db") # 파일로 저장

In [19]:
chroma_db._collection.count()

646

In [20]:
# 데이터 저장 구조 (구성) 확인
# for k, v in chroma_db._collection.get().items(): # items() : key, value from dict
#     print(k)

for k in chroma_db._collection.get(): # key from dict
    print(k)

ids
embeddings
documents
uris
data
metadatas
included


In [21]:
chroma_db._collection.get()["ids"] # 데이터 반환
chroma_db._collection.get()["embeddings"] # 어떤 데이터는 None으로 반환
chroma_db._collection.get(include=["embeddings"])['embeddings']

array([[-0.0077088 , -0.0075166 ,  0.014182  , ..., -0.01861645,
        -0.00669286, -0.00903364],
       [ 0.00170697, -0.01484938,  0.03096656, ...,  0.00016448,
        -0.00243612, -0.01603626],
       [-0.01100211, -0.00490143,  0.03516901, ..., -0.00774796,
         0.00382345, -0.02089929],
       ...,
       [ 0.00093808, -0.02087439,  0.05270585, ..., -0.01743532,
        -0.00393228, -0.00537856],
       [-0.01949805, -0.03509107,  0.01503709, ..., -0.00687449,
        -0.00983716, -0.01018292],
       [-0.0089351 , -0.02862453,  0.02305268, ...,  0.00560877,
        -0.00409497, -0.00419903]])

In [22]:
embeddings = chroma_db._collection.get(include=["embeddings"])['embeddings']

In [23]:
embeddings.shape

(646, 1536)

In [24]:
# 유사도가 높은 2개의 문서를 반환하는 반환기로 설정
retriever = chroma_db.as_retriever(search_kwargs={"k":2}) 

In [25]:
docs = retriever.get_relevant_documents("비트코인을 알려주세요")

C:\Users\human\AppData\Local\Temp\ipykernel_11860\73732550.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents("비트코인을 알려주세요")


In [26]:
print(len(docs))
print(docs[0])

2
page_content='139
ㅂ 
비트코인
비트코인(bitcoin)은 가상통화(암호통화)이자 디지털 지급시스템이다. 비트코인 시스템
은 중앙 저장소 또는 단일 관리자가 없기 때문에 최초의 탈중앙화된 디지털통화라고 불린다. 
이는 사토시 나카모토라는 사람(집단)에 의해 만들어져서 2009년 개방형 소프트웨어로 
배포되었다. 이 시스템은 공유형(peer-to-peer)이며, 거래는 중개자 없이 블록체인 소프트
웨어를 이용하는 참여자(nodes) 사이에 직접 이뤄진다. 이런 거래들은 공유(P2P) 네트워크
상 참여자의 작업증명(proof-of-work)을 통해 검증되고 공개된 분산원장인 블록체인에 
기록된다. 승인된 거래의 새 그룹인 한 블록은 대략 10분에 한 번씩 만들어져서 블록체인에 
추가되고 신속하게 모든 참여자에게 보내어진다. 비트코인은 대규모 전기를 사용하는 
컴퓨터 처리 능력(power)을 활용해서 이뤄지는 기록보관 서비스인 채굴(mining)에 대한 
보상으로 발행되고 있으며 다른 통화･상품･용역 등과 교환되어가고 있다. 중앙은행이 
발행한 법정화폐가 아닌 비트코인은 비트코인 플랫폼에서 거래되며 투자대상으로서도 
관심을 받고 있다. 하지만 급등한 가격 및 심한 변동 폭으로 거품논란이 크다. 또한 익명성으
로 자금세탁 등 불법거래에 악용될 소지가 많아 중국 등 일부 국가에서 비트코인 등 가상통
화의 거래를 규제하고 있다. 일본의 경우 비트코인의 거래이익에 대해 세금을 부과한다. 
비트코인은 추가되는 한 블록당 새로 12.5비트코인을 보상하는데(2016.7월 현재), 21만개
가 채굴될 때(대략 4년)마다 그 보상이 반으로 줄어든다. 비트코인의 총량은 21백만개이며 
2140년경 모두 채굴될 것으로 전망된다. 비트코인은 그 시스템의 설계시 그 수량을 한정시
켜 놓았기 때문에 원칙적으로 인플레이션에 의해 가치가 떨어질 수 있는 화폐와 다른 
속성을 지닌다. 한편 2017년 8월 1일 비트코인(classic bitcoin)에서 ‘비트코인캐시’(BCH)

In [27]:
# 템플릿 만들기
template = """
당신은 한국은행에서 만든 금융 용어를 설명해주는 도우미입니다.
주어진 검색 결과를 기반으로 답변해 주세요.
검색 결과에 없는 내용은 답변할 수 없다고 해주세요.

[검색결과]
{context}

질문: {question}
답변:"""

prompt = PromptTemplate.from_template(template=template)

In [28]:
llm = ChatOpenAI(model="gpt-4o", temperature=0)

In [29]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type_kwargs={"prompt": prompt},
    retriever=retriever,
    return_source_documents=True
)

In [30]:
response = qa_chain.invoke("비트코인은 무엇인가요?")
response

{'query': '비트코인은 무엇인가요?',
 'result': '비트코인(bitcoin)은 가상통화(암호통화)이자 디지털 지급시스템으로, 중앙 저장소나 단일 관리자가 없는 최초의 탈중앙화된 디지털 통화입니다. 2009년 사토시 나카모토라는 사람(또는 집단)에 의해 개방형 소프트웨어로 배포되었습니다. 비트코인 시스템은 공유형(peer-to-peer) 네트워크를 통해 중개자 없이 거래가 이루어지며, 거래는 참여자(nodes) 사이에서 블록체인 소프트웨어를 이용해 직접 검증되고 기록됩니다. 비트코인은 채굴(mining)이라는 과정을 통해 발행되며, 다른 통화, 상품, 용역 등과 교환될 수 있습니다. 비트코인은 중앙은행이 발행한 법정화폐가 아니며, 투자 대상으로도 관심을 받고 있습니다. 그러나 가격의 급등과 변동성으로 인해 거품 논란이 있으며, 익명성으로 인해 자금세탁 등 불법 거래에 악용될 가능성이 있어 일부 국가에서는 거래를 규제하고 있습니다. 비트코인의 총량은 2100만 개로 한정되어 있으며, 2140년경 모두 채굴될 것으로 예상됩니다.',
 'source_documents': [Document(metadata={'page': 155, 'source': 'data-files/economy-dictionary.pdf'}, page_content='139\nㅂ \n비트코인\n비트코인(bitcoin)은 가상통화(암호통화)이자 디지털 지급시스템이다. 비트코인 시스템\n은 중앙 저장소 또는 단일 관리자가 없기 때문에 최초의 탈중앙화된 디지털통화라고 불린다. \n이는 사토시 나카모토라는 사람(집단)에 의해 만들어져서 2009년 개방형 소프트웨어로 \n배포되었다. 이 시스템은 공유형(peer-to-peer)이며, 거래는 중개자 없이 블록체인 소프트\n웨어를 이용하는 참여자(nodes) 사이에 직접 이뤄진다. 이런 거래들은 공유(P2P) 네트워크\n상 참여자의 작업증명(proof-of-work)을 통해 검증되고 공개된 분산원장인 블록체인에 \n기록된다. 승인된 거래의 새 

In [31]:
response = qa_chain.invoke("너 누구야?")
response

{'query': '너 누구야?',
 'result': '죄송하지만, 주어진 검색 결과에 해당하는 정보가 없습니다. 다른 질문을 해주시면 도움이 될 수 있는지 확인해보겠습니다.',
 'source_documents': [Document(metadata={'page': 274, 'source': 'data-files/economy-dictionary.pdf'}, page_content='① 한 사람이 자백하고 다른 사람은 입을 다물었다면 자백한 사람은 1단위의 보상 , \n입을 다문 사람은 2단위의 벌금형이 부과된다. ② 두 사람이 모두 자백하면 각각에게는 \n1단위의 벌금형이 부과된다. ③ 두 사람이 모두 자백하지 않으면 Ⅰ과 Ⅱ는 혐의가'),
  Document(metadata={'page': 306, 'source': 'data-files/economy-dictionary.pdf'}, page_content='달리는 차가 충돌하기 직전에 운전대를 바꾸는 사람이 겁쟁이(chicken)로 놀림을 받았다\n고 한다. 이러한 종류의 게임에서 승리하기 위해서는 상대방에게 자신의 선택을 절대로 \n바꾸지 않겠다는 의지를 정확히 전달하는 것이 매우 중요하다. 그러나 이해당사자가 \n모두 자신에게 유리한 선택만을 고집할 경우 모두에게 최악의 결과가 도래하게 된다 . \n게임 전문가들은 치킨게임이 이해당사자들 간에 반복될 경우 이론적으로는 자신에게 \n가장 유리한 선택을 포기하게 됨으로써 최악의 상황에 도달하지는 않는다고 한다 . \n 연관검색어 : 죄수의 딜레마')]}

In [33]:
# !pip install gradio==4.44.1

In [34]:
def get_chatbot_response(message):
    chatbot_response = qa_chain.invoke(message)
    return chatbot_response["result"].strip()

In [35]:
import gradio as gr

# 인터페이스를 생성.
with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="경제금융용어 챗봇") # 경제금융용어 챗봇 레이블을 좌측 상단에 구성
    msg = gr.Textbox(label="질문해주세요!")  # 하단의 채팅창의 레이블
    clear = gr.Button("대화 초기화")  # 대화 초기화 버튼

    # 챗봇의 답변을 처리하는 함수
    def respond(message, chat_history):
      bot_message = get_chatbot_response(message)

      # 채팅 기록에 사용자의 메시지와 봇의 응답을 추가.
      chat_history.append((message, bot_message))
      return "", chat_history

    # 사용자의 입력을 제출(submit)하면 respond 함수가 호출.
    msg.submit(respond, [msg, chatbot], [msg, chatbot])

    # '초기화' 버튼을 클릭하면 채팅 기록을 초기화.
    clear.click(lambda: None, None, chatbot, queue=False)

# 인터페이스 실행.
demo.launch(debug=True)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.
